# Reading Whole-slide Images using Large-image

This notebook demonstrates how to use [Large-image](https://github.com/girder/large_image) to seamlessly read different kinds of information from large multi-resolution whole-slide images. 

In [1]:
import large_image

INFO:girder:Created LRU Cache for 'tilesource' with 335 maximum size
INFO:root:Notice: Could not import MapnikTileSource


In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

#Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['image.cmap'] = 'gray'

#### Download a sample whole-slide image

The following code downloads a sample slide from data.kitware.com

In [3]:
wsi_path = 'TCGA-02-0010-01Z-00-DX4.07de2e55-a8fe-40ee-9e98-bcb78050b9f7.svs'

if not os.path.isfile(wsi_path):
    !curl -OJ 'https://data.kitware.com/api/v1/file/5899dd6d8d777f07219fcb23/download'

#### Loading a whole-slide image

In [4]:
ts = large_image.getTileSource(wsi_path)

INFO:girder:Using memcached for large_image caching


Using memcached for large_image caching


The `getTileSource()` function returns an object of the TileSource class that contains the following utility functions for reading different kinds of information from whole-slide images:

* [getMetadata()](#getMetadata) - Gets basic slide metadata
* [getNativeMagnication()](#getNativeMagnification) - Gets the magnification at which the slide was scanned
* [getMagnicationForLevel()](#getMagnificationForLevel) - Gets the magnification of the specified level in the image pyramid
* [getLevelForMagnification()](#getLevelForMagnification) - Gets the level of the image pyramid closest to a specified magnification
* [tileIterator()](#tileIterator) - Gets an iterator for sequentially iterating through the slide in a tiled fashion at any scale/magnification
* [getSingleTile()](#getSingleTile) - Gets the tile at a specified linear position of the tileIterator.
* [getRegion()](#getRegion) - Gets a region of interest (ROI) within the slide at a specified scale.
* [convertRegionScale()](#convertRegionScale) - Converts a region from one scale to another.
* [getRegionAtAnotherScale()](#getRegionAtAnotherScale) - Gets an image of a region at another scale.

The purpose and typical usage of each of these utility functions is presented in detail below.

<a id="getMetadata"></a>

#### getMetadata()

The `getMetadata()` function of the TileSource class returns a python dict containing basic metadata of a slide:

* *levels* - number of levels in the image pyramid
* *magnification* - native magnification or the magnification associated with the base or highest resolution level at which the slide was scanned.
* *mm_x, mm_y* - width and height of a pixel in millimeters at the base resolution. 
* *sizeX, sizeY* - width and height of the slide in pixels at the base resolution.
* *tileWidth, tileHeight* - width and height of a tile in pixels at the base resolution.

In [5]:
ts.getMetadata()

{'levels': 9,
 'magnification': 20.0,
 'mm_x': 0.0005015,
 'mm_y': 0.0005015,
 'sizeX': 32001,
 'sizeY': 38474,
 'tileHeight': 240,
 'tileWidth': 240}

<a id="getNativeMagnification"></a>

#### getNativeMagnification()

The `getNativeMagnification()` function of the TileSource class returns a python dict containing the magnification and physical size of a pixel in millimeters at the base or highest resolution level at which the slide was scanned

In [6]:
ts.getNativeMagnification()

{'magnification': 20.0, 'mm_x': 0.0005015, 'mm_y': 0.0005015}

<a id="getMagnificationForLevel"></a>

#### getMagnificationForLevel()

The `getMagnificationForLevel()` function of the TileSource class returns a python dict containing the magnification and physical size of a pixel for a specified level in the image pyramid.

In [7]:
# Get the magnification associated with Level 0
ts.getMagnificationForLevel(level=0)

{'level': 0,
 'magnification': 0.078125,
 'mm_x': 0.128384,
 'mm_y': 0.128384,
 'scale': 256.0}

In [8]:
# Get the magnification associated with all levels of the image pyramid
for i in range(ts.levels):
    print('Level-{} : {}'.format(i, ts.getMagnificationForLevel(level=i)))

Level-0 : {'magnification': 0.078125, 'scale': 256.0, 'level': 0, 'mm_y': 0.128384, 'mm_x': 0.128384}
Level-1 : {'magnification': 0.15625, 'scale': 128.0, 'level': 1, 'mm_y': 0.064192, 'mm_x': 0.064192}
Level-2 : {'magnification': 0.3125, 'scale': 64.0, 'level': 2, 'mm_y': 0.032096, 'mm_x': 0.032096}
Level-3 : {'magnification': 0.625, 'scale': 32.0, 'level': 3, 'mm_y': 0.016048, 'mm_x': 0.016048}
Level-4 : {'magnification': 1.25, 'scale': 16.0, 'level': 4, 'mm_y': 0.008024, 'mm_x': 0.008024}
Level-5 : {'magnification': 2.5, 'scale': 8.0, 'level': 5, 'mm_y': 0.004012, 'mm_x': 0.004012}
Level-6 : {'magnification': 5.0, 'scale': 4.0, 'level': 6, 'mm_y': 0.002006, 'mm_x': 0.002006}
Level-7 : {'magnification': 10.0, 'scale': 2.0, 'level': 7, 'mm_y': 0.001003, 'mm_x': 0.001003}
Level-8 : {'magnification': 20.0, 'level': 8, 'scale': 1.0, 'mm_y': 0.0005015, 'mm_x': 0.0005015}


<a id="getLevelForMagnification"></a>

#### getLevelForMagnification()

The `getLevelForMagnification()` function of the TileSource class returns the level of the image pyramid associated with a specific magnification or pixel size in millimeters.

In [9]:
# get level whose magnification is closest to 10x
print('Level with magnification closest to 10x = {}'.format(ts.getLevelForMagnification(10)))

Level with magnification closest to 10x = 7


In [10]:
# get level whose pixel width is closest to 0.0005 mm
print('Level with pixel width closest to 0.0005mm = {}'.format(ts.getLevelForMagnification(mm_x=0.0005)))

Level with pixel width closest to 0.0005mm = 8


<a id="tileIterator"></a>

#### tileIterator()

The `tileIterator()` function provides a iterator for sequentially iterating through the entire slide or a region of interest (ROI) within the slide at any desired resolution in a tile-wise fashion. 

Among others, below are the main optional parameters of tileIterator that cover most of the tile-wise iteration use-cases for image analysis:

* `region` - allows you to specify an ROI within the slide.
* `scale` - allows you to specify the desired magnification/resolution.
* `tile_size` - allows you to specify the size of the tile.
* `tile_overlap` - allows you to specify the amount of overlap between adjacent tiles.
* `format` - allows you to specify the format of the tile image (numpy array or PIL image).

At each iteration the tileIterator outputs a dictionary that includes:

* `tile` - cropped tile image that is lazy loaded or computed only when this element of the dictionary is explicitly accessed.
* `format` - format of the tile.
* `x, y` - (left, top) coordinates in current magnification pixels.
* `width, height` -  size of current tile in current magnification pixels.
* `level` - level of the current tile.
* `magnification` - magnification of the current tile.
* `mm_x, mm_y` - size of the current tile pixel in millimeters.
* `gx, gy` - (left, top) coordinate in base/maximum resolution pixels.
* `gwidth, gheight` - size of of the current tile in base/maximum resolution pixels.

The lazy loading of the tile image allows us to quickly iterate through the tiles and selectively process tiles of interest based on the tile metadata.

Take a look at the help of this function for a detailed description of the inputs, parameters and outputs

In [ ]:
help(ts.tileIterator)

Help on method tileIterator in module server.tilesource.base:

tileIterator(self, format=('numpy',), resample=False, **kwargs) method of server.tilesource.svs.SVSFileTileSource instance
    Iterate on all tiles in the specifed region at the specified scale.
    Each tile is returned as part of a dictionary that includes
        x, y: (left, top) coordinates in current magnification pixels
        width, height: size of current tile incurrent magnification pixels
        tile: cropped tile image
        format: format of the tile
        level: level of the current tile
        level_x, level_y: the tile reference number within the level.
        magnification: magnification of the current tile
        mm_x, mm_y: size of the current tile pixel in millimeters.
        gx, gy: (left, top) coordinate in maximum-resolution pixels
        gwidth, gheight: size of of the current tile in maximum resolution
            pixels.
    If a region that includes partial tiles is requested, those til

The code below shows how to iterate through an ROI within a slide with a specific tile size and at a specific resolution

In [ ]:
num_tiles = 0

tile_means = []
tile_areas = []

for tile_info in ts.tileIterator(region=dict(left=5000, top=5000, width=20000, height=20000, units='base_pixels'), 
                                 scale=dict(magnification=20),
                                 tile_size=dict(width=1000, height=1000), 
                                 tile_overlap=dict(x=50, y=50),
                                 format=large_image.tilesource.TILE_FORMAT_PIL
                                ):
    
    if num_tiles == 100:
        print('Tile-{} = '.format(num_tiles))
        display(tile_info)
        
    im_tile = np.array(tile_info['tile'])
    tile_mean_rgb = np.mean(im_tile[:, :, :3], axis=(0, 1))
    
    tile_means.append( tile_mean_rgb )
    tile_areas.append( tile_info['width'] * tile_info['height'] )
    
    num_tiles += 1
    
slide_mean_rgb = np.average(tile_means, axis=0, weights=tile_areas)

print('Number of tiles = {}'.format(num_tiles))
print('Slide mean color = {}'.format(slide_mean_rgb))

Tile-100 = 


{'format': 'PIL',
 'gheight': 1000.0,
 'gwidth': 1000.0,
 'gx': 16150.0,
 'gy': 8550.0,
 'height': 1000,
 'iterator_range': {'level_x_max': 27,
  'level_x_min': 5,
  'level_y_max': 27,
  'level_y_min': 5,
  'position': 484,
  'region_x_max': 22,
  'region_y_max': 22},
 'level': 8,
 'level_x': 17,
 'level_y': 9,
 'magnification': 20.0,
 'mm_x': 0.0005015,
 'mm_y': 0.0005015,
 'tile': <PIL.Image.Image image mode=RGBA size=1000x1000 at 0x7F3A025C0850>,
 'tile_overlap': {'bottom': 25, 'left': 25, 'right': 25, 'top': 25},
 'tile_position': {'level_x': 17,
  'level_y': 9,
  'position': 100,
  'region_x': 12,
  'region_y': 4},
 'width': 1000,
 'x': 16150,
 'y': 8550}

<a id="getSingleTile"></a>

#### getSingleTile()

The `getSingleTile()` function can be used to directly get the tile at a specific position of the tile iterator. 
In addition to the aformentioned parameters of the `tileIterator`, it takes a tile_position parameter that can be used to specify the linear position of the tile of interest. 

In [ ]:
pos = 1000

tile_info = ts.getSingleTile(tile_size=dict(width=1000, height=1000), 
                             scale=dict(magnification=20), 
                             tile_position=pos)

plt.imshow(tile_info['tile'])

<a id="getRegion"></a>

#### getRegion()

The `getRegion()` function can be used to get a rectangular region of interest (ROI) within the slide at any scale/magnification via the following two parameters:

* `region` - a dictionary containing the (left, top, width, height, units) of the ROI
* `scale` - a dictionary containing the magnification or the physical size of a pixel (mm_x, mm_y)

The following code reads the entire slide at a low magnification

In [ ]:
im_low_res, _ = ts.getRegion(scale=dict(magnification=1.25), 
                             format=large_image.tilesource.TILE_FORMAT_NUMPY)

plt.imshow(im_low_res)

The following code shows how to read an ROI

In [ ]:
im_roi, _ = ts.getRegion(region=dict(left=10000, top=10000, width=1000, height=1000, units='base_pixels'),
                         format=large_image.tilesource.TILE_FORMAT_NUMPY)

plt.imshow(im_roi)

<a id="convertRegionScale"></a>

#### convertRegionScale()

The `convertRegionScale()` function can be used to convert a region from one scale/magnification to another as illustrated in the following example

In [ ]:
help(ts.convertRegionScale)

In [ ]:
tr = ts.convertRegionScale(sourceRegion=dict(left=5000, top=5000, width=1000, height=1000, units='mag_pixels'), 
                           sourceScale=dict(magnification=20),
                           targetScale=dict(magnification=10),
                           targetUnits='mag_pixels'
                          )

display(tr)

<a id="getRegionAtAnotherScale"></a>

#### getRegionAtAnotherScale()

The getRegionAtAnotherScale() function can be used to get an image of a region at another scale.

In [ ]:
help(ts.getRegionAtAnotherScale)

In [ ]:
# get a large region defined at base resolution at a much lower scale
im_roi, _ = ts.getRegionAtAnotherScale(
    sourceRegion=dict(left=5000, top=5000, width=10000, height=10000, units='base_pixels'),
    targetScale=dict(magnification=1.25),
    format=large_image.tilesource.TILE_FORMAT_NUMPY)

print im_roi.shape